# Image Detection Demo with PytorchWildlife

This tutorial guides you on how to use PyTorchWildlife to separate positive and negative animal detections. We will go through the process of setting up the environment, defining the detection model, as well as performing inference and saving the results in different ways.

## Prerequisites
Install PytorchWildlife running the following commands:
```bash
conda create -n pytorch_wildlife python=3.8 -y
conda activate pytorch_wildlife
pip install PytorchWildlife
```
Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. This notebook can also run on CPU.

## Importing libraries
First, we'll start by importing the necessary libraries and modules.

In [8]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [9]:
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

## Model Initialization
We will initialize the MegaDetectorV5 model for image detection. This model is designed for detecting animals in images.

In [10]:
# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)
    
# Initializing the MegaDetectorV6 model for image detection
#detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="yolov9c")

# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


## Variable definition
In order to process the batch detection, we will define an input directory where the images are stored, a confidence threshold and an output directory to copy the positive and negative images into distinctive folders. If you want to follow this tutorial with your own data, modify the following variables.

In [12]:
#tgt_folder_path = os.path.join(".","demo_data","imgs")
tgt_folder_path = "/media/mo/nvme0n1/PW_MDv5_v6_test_HG_input/Q16"

#output_path = "folder_separation"
output_path = "/media/mo/nvme0n1/PW_MDv5a_test_HG_separated_subfolder"
#output_path = "/media/mo/nvme0n1/MDv5b_test_HG_separated"

#output_path = "/media/mo/nvme0n1/MDv6_test_HG_separated"
#output_path = "/media/mo/nvme0n1/MDv6_test_HG_separated_subfolder"
threshold = 0.1 #threshold = 0.1




## Batch Image Detection
Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.

In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
results = detection_model.batch_image_detection(tgt_folder_path, batch_size=5) #default batch_size=16, highVRAM,failed= 350, v5a= 400, v6= 

100%|██████████| 1289/1289 [05:02<00:00,  4.25it/s]


## Separate positive and negative detections
PytorchWildlife allows to copy the files from your original folder to a new directory containing the "Animal" and "No-animal" subdirectories. A detection is considered positive if the prediction confidence is higher than the threshold

In [7]:
os.makedirs(output_path, exist_ok=True)
json_file = os.path.join(output_path, "detection_results.json")
pw_utils.save_detection_json(results, json_file,
                             categories=detection_model.CLASS_NAMES,
                             exclude_category_ids=[], # Category IDs can be found in the definition of each model.
                             exclude_file_path=None) #exclude_file_path=tgt_folder_path)

# Separate the positive and negative detections through file copying:
pw_utils.detection_folder_separation(json_file, tgt_folder_path, output_path, threshold)



SameFileError: '/media/mo/nvme0n1/MDv5_v6_test_HG/Q18/CAM141/20230725/100_BTCF/IMG_0001.JPG' and '/media/mo/nvme0n1/MDv5_v6_test_HG/Q18/CAM141/20230725/100_BTCF/IMG_0001.JPG' are the same file

In [15]:
#type of the json_file, what is it?
type(results)
#head of the results
results[0:1]


    


[{'img_id': '/media/mo/nvme0n1/MDv5_v6_test_HG/Q18/CAM141/20230725/100_BTCF/IMG_0001.JPG',
  'detections': Detections(xyxy=array([[       1329,         471,        1456,         585]], dtype=float32), mask=None, confidence=array([    0.26743], dtype=float32), class_id=array([0]), tracker_id=None, data={}),
  'labels': ['animal 0.27'],
  'normalized_coords': [[0.6921875,
    0.4361111111111111,
    0.7583333333333333,
    0.5416666666666666]]}]

### Copyright (c) Microsoft Corporation. All rights reserved.
### Licensed under the MIT License.